In [1]:
import torch
import torch.nn as nn
import time
import os

## Load model

In [2]:
class FCN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FCN, self).__init__()
        self.input_size = input_size
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out) 
        return out

In [3]:
model = FCN(1,1,1)
model = torch.load('full_model.pth')
model

FCN(
  (l1): Linear(in_features=36, out_features=25, bias=True)
  (relu): ReLU()
  (l2): Linear(in_features=25, out_features=5, bias=True)
)

In [4]:
for weight in model.state_dict():
    print(weight)

l1.weight
l1.bias
l2.weight
l2.bias


In [5]:
for key, val in model.state_dict().items():
    print(key, val.shape)

l1.weight torch.Size([25, 36])
l1.bias torch.Size([25])
l2.weight torch.Size([5, 25])
l2.bias torch.Size([5])


## Print weights for HLS

In [6]:
l1_weight = model.state_dict()['l1.weight'].tolist()
l2_weight = model.state_dict()['l2.weight'].tolist()
l1_bias = model.state_dict()['l1.bias'].tolist()
l2_bias = model.state_dict()['l2.bias'].tolist()

In [7]:
print("const float l1_bias[HIDDEN_SIZE] = {", end='')
biases = ''
for b in l1_bias:
    biases += str(b) + ', '
biases = biases[:-2] + '};'
print(biases)

const float l1_bias[HIDDEN_SIZE] = {-0.12587140500545502, -0.030282268300652504, -0.21604034304618835, -0.13532671332359314, -0.16997180879116058, 0.08291299641132355, -0.2593229115009308, -0.10419173538684845, 4.092952728271484, 0.037874456495046616, 0.11178559064865112, -9.537230491638184, 8.764646530151367, -0.19718784093856812, -0.2631480097770691, -3.8321239948272705, 0.032348811626434326, 0.02701716311275959, 0.019858315587043762, 0.06674651801586151, 2.7918968200683594, -0.1919824331998825, -0.1855362504720688, -0.2080673724412918, 0.010033066384494305};


In [8]:
print("const float l2_bias[NUM_CLASSES] = {", end='')
biases = ''
for b in l2_bias:
    biases += str(b) + ', '
biases = biases[:-2] + '};'
print(biases)

const float l2_bias[NUM_CLASSES] = {1.867798089981079, -4.117910385131836, -5.021473407745361, -1.7539169788360596, 7.344239711761475};


In [9]:
print("const float l1_weights[HIDDEN_SIZE][INPUT_SIZE] = {", end='')
weights = ''
for row in l1_weight:
    weights += '{'
    for cell in row:
        weights += str(cell) + ', '
    weights = weights[:-2] + '},\n'
weights = weights[:-2] + '};'
print(weights)

const float l1_weights[HIDDEN_SIZE][INPUT_SIZE] = {{-0.1913929581642151, 0.2055724412202835, -0.044019535183906555, -0.020334608852863312, 0.07787514477968216, 0.09279234707355499, 0.09967907518148422, 0.13256751000881195, 0.026141032576560974, 0.05092580243945122, 0.0056466348469257355, -0.018013067543506622, -0.014142489060759544, -0.07350168377161026, -0.026758790016174316, 0.07881755381822586, -0.2137240618467331, -0.18719710409641266, 0.00111151032615453, -0.014146171510219574, 0.12850043177604675, -0.19412626326084137, 0.10198605060577393, -0.047522254288196564, 0.03064282424747944, 0.05807548016309738, -0.06901183724403381, 0.009364143945276737, -0.03802873566746712, -0.1875050663948059, -0.06441982835531235, 0.1378535032272339, 0.0713295191526413, -0.2081601768732071, -0.01907765306532383, 0.01469804160296917},
{-0.055784858763217926, -0.061334989964962006, 0.08690322190523148, 0.09142344444990158, -0.06811029464006424, -0.09958883374929428, 0.03342488408088684, 0.1871233135461

In [10]:
print("const float l2_weights[NUM_CLASSES][HIDDEN_SIZE] = {", end='')
weights = ''
for row in l2_weight:
    weights += '{'
    for cell in row:
        weights += str(cell) + ', '
    weights = weights[:-2] + '},\n'
weights = weights[:-2] + '};'
print(weights)

const float l2_weights[NUM_CLASSES][HIDDEN_SIZE] = {{0.013823124580085278, 0.12248829752206802, -0.1322016417980194, 0.03134175017476082, 0.16245828568935394, 0.09732319414615631, 0.06483464688062668, -0.17047066986560822, 0.3751170039176941, 0.06981124728918076, -0.012196303345263004, -0.4167352616786957, 0.5107572674751282, 0.11999634653329849, -0.1740768700838089, 0.06722865998744965, -0.00034535396844148636, 0.09567374736070633, 0.06403892487287521, 0.04089426249265671, 0.16087165474891663, -0.050999727100133896, 0.1494142711162567, 0.13268695771694183, 0.1997201293706894},
{0.1230154037475586, 0.09777972847223282, -0.12872935831546783, -0.21899710595607758, -0.20308618247509003, -0.11968117207288742, 0.14648683369159698, -0.08010698109865189, -0.6130756139755249, 0.12246651202440262, -0.03719998151063919, 0.12846070528030396, -0.5385376214981079, 0.11546987295150757, -0.16755259037017822, -0.0850461795926094, 0.09287487715482712, -0.050956904888153076, 0.1371791660785675, 0.153020

## Get test data for software-hardware comparison

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import preprocessing

In [12]:
DATA_DIR = 'data'
RAW = DATA_DIR + '/raw'
NUM_FEATURES = 36

In [13]:
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
X_test = test_df.drop('action', axis='columns')
y_test = test_df['action']

In [14]:
data = [torch.Tensor(X_test.iloc[x].astype(float)) for x in range(5)]

In [15]:
data

[tensor([ 5.2000e-01, -4.0000e-01,  1.1325e-01,  1.9203e-01,  2.2750e-01,
          1.4825e-01,  1.4100e+00, -9.8000e-01,  1.0400e-01,  4.3644e-01,
          2.7250e-01,  2.8330e-01,  0.0000e+00, -5.2000e-01, -2.6600e-01,
          1.2907e-01,  1.2750e-01,  9.7000e-02,  1.5656e+02, -7.1330e+01,
          5.9300e+00,  3.7764e+01,  3.4450e+01,  2.5990e+01,  1.5781e+02,
         -1.2462e+02,  3.1607e+00,  5.7565e+01,  6.2780e+01,  4.2847e+01,
          2.0589e+02, -2.6263e+02, -9.5075e-01,  6.9546e+01,  3.9938e+01,
          4.2093e+01]),
 tensor([ 1.3700e+00, -1.9100e+00, -1.6200e-01,  6.7358e-01,  3.1250e-01,
          4.4660e-01,  2.4600e+00, -8.8000e-01,  3.8450e-01,  8.0625e-01,
          2.5250e-01,  5.9775e-01,  5.2000e-01, -3.4000e-01, -5.2500e-02,
          1.6392e-01,  1.5500e-01,  1.0950e-01,  1.4823e+02, -3.1834e+02,
         -3.4742e+01,  9.6137e+01,  4.8138e+01,  6.9615e+01,  2.7069e+02,
         -2.6349e+02,  1.0544e+01,  1.1798e+02,  6.3443e+01,  8.0055e+01,
          2.84

In [16]:
with torch.no_grad():
    for d in data:
        start_time = time.time()
        output = model(d)
        action = np.argmax(output)
#         print(f"Inference time: {time.time() - start_time}")    
        print(action)

tensor(4)
tensor(1)
tensor(0)
tensor(3)
tensor(3)


In [17]:
for x in range(5):
    print(y_test[x])

4
1
0
3
3


## Compare software-hardware running time and accuracy

In [18]:
# 1 sample

with torch.no_grad():
    start_time = time.time()

    data = torch.Tensor(X_test.iloc[0].astype(float))
    output = model(data)
    action = np.argmax(output)

    end_time = time.time()
    print(f"Prediction: {action}; Ground truth value: {y_test[0]}")
    print(f"Inference time: {((end_time - start_time) * 1000):.4f} ms") 

Prediction: 4; Ground truth value: 4
Inference time: 0.6721 ms


In [19]:
# whole test set (or dev + test set used in training)

with torch.no_grad():
    start_time = time.time()
    
    num_correct = 0
    num_samples = 0

    for i in range(len(X_test[:])):
        data = torch.Tensor(X_test.iloc[i].astype(float))
        output = model(data)
        action = np.argmax(output)
        num_samples += 1
        num_correct += (action == y_test[i])

    end_time = time.time()
    print(f"Accuracy: {(100.0 * num_correct / num_samples):.2f}%")
    print(f"Inference time: {(end_time - start_time):.4f} s")    

Accuracy: 96.14%
Inference time: 0.1286 s


In [20]:
len(X_test)

285